In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gc # Garbage Collector
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# 1. LOAD TARGET
train_df = pd.read_csv('train.csv', usecols=['exam_score'])
y_true = train_df['exam_score'].values.astype(np.float32)
del train_df
gc.collect()

# 2. FIND MODELS
oof_files = sorted(glob.glob("*_oof.csv"))
sub_files = [f.replace("_oof.csv", "_sub.csv") for f in oof_files]
print(f"Ensembling {len(oof_files)} models...")

# 3. LOAD DATA IN FLOAT32 (Saves 50% RAM)
oofs = []
subs = []

for oof_f, sub_f in zip(oof_files, sub_files):
    if os.path.exists(sub_f):
        # Load only the column we need and convert to float32 immediately
        oofs.append(pd.read_csv(oof_f, usecols=['exam_score'])['exam_score'].values.astype(np.float32))
        subs.append(pd.read_csv(sub_f, usecols=['exam_score'])['exam_score'].values.astype(np.float32))
        # Clear small temp buffers
        gc.collect()

X_train_stack = np.stack(oofs, axis=1)
X_test_stack = np.stack(subs, axis=1)

# Clear the lists to free up memory
del oofs, subs
gc.collect()

print(f"Stacking matrix created. Shape: {X_train_stack.shape}")

# 4. RIDGE STACKING (Optimized)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
# Reduced alpha range to save computation
alphas = np.logspace(0, 5, 20) 

oof_final_preds = np.zeros(len(y_true), dtype=np.float32)
sub_final_preds = np.zeros(X_test_stack.shape[0], dtype=np.float32)

print("\n--- Starting Ridge Stacking ---")
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_stack)):
    X_tr, y_tr = X_train_stack[train_idx], y_true[train_idx]
    X_va, y_va = X_train_stack[val_idx], y_true[val_idx]
    
    # We use the RidgeCV but restrict it to one core (n_jobs=1) 
    # if it still crashes, or use n_jobs=-1 for speed if RAM is okay.
    model = RidgeCV(alphas=alphas) 
    model.fit(X_tr, y_tr)
    
    oof_final_preds[val_idx] = model.predict(X_va)
    sub_final_preds += model.predict(X_test_stack) / 10
    
    print(f"Fold {fold+1} complete. RMSE: {rmse(y_va, oof_final_preds[val_idx]):.6f}")
    gc.collect()

# 5. FINAL SUBMISSION
final_sub_preds = np.clip(sub_final_preds, 0, 100)
submission = pd.read_csv('sample_submission.csv')
submission['exam_score'] = final_sub_preds
submission.to_csv('super_submission.csv', index=False)

print(f"\nFinal Combined OOF RMSE: {rmse(y_true, oof_final_preds):.6f}")

Ensembling 40 models...
Stacking matrix created. Shape: (630000, 40)

--- Starting Ridge Stacking ---
Fold 1 complete. RMSE: 8.550329
Fold 2 complete. RMSE: 8.606216
Fold 3 complete. RMSE: 8.552633
Fold 4 complete. RMSE: 8.614249
Fold 5 complete. RMSE: 8.559106
Fold 6 complete. RMSE: 8.592434
Fold 7 complete. RMSE: 8.611374
Fold 8 complete. RMSE: 8.557966
Fold 9 complete. RMSE: 8.594989
Fold 10 complete. RMSE: 8.613421

Final Combined OOF RMSE: 8.585311
